# 1. Entorno

In [1]:
#Creamos un Notebook en Python sobre JupyterLab

In [2]:
#Instalamos la librerías para S3
!pip install s3fs

  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec None
error: uninstall-no-record-file

× Cannot uninstall fsspec None
╰─> The package's contents are unknown: no RECORD file was found for fsspec.

hint: You might be able to recover from this via: pip install --force-reinstall --no-deps fsspec==2023.6.0


# 2. Librerías

In [1]:
#Utilitario para construir modelos de Machine Learning y Deep Learning
import sagemaker

#Librería para manipular los servicios de AWS
import boto3

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


# 3. Inicio de Sesión sobre SageMaker

In [3]:
#Iniciamos sesión en el servicio de SageMaker
sesion = sagemaker.Session()

#Obtenemos la ejecución en donde estamos trabajando
region = sesion.boto_region_name

#Obtenemos el rol de ejecución de SageMaker
#El rol de ejecución permite a SageMaker usar otros servicios de AWS
rol = sagemaker.get_execution_role()

# 4. Lectura de archivo de casos

In [4]:
#Importamos la versión clásica de Pandas
#Pandas no procesará los datos, sólo los leerá para enviarselos al modelo, no necesitamos la versión de Big Data
import pandas as pd

In [5]:
#Nos conectamos al servicio de S3
s3 = boto3.client("s3")

In [6]:
#Bucket de almacenamiento
#REEMPLAZAMOS XXX POR NUESTRAS INICIALES
bucket = "datasetsbdajac"

In [7]:
#Utilitario para manipular el sistema de archivos de S3
import s3fs

In [8]:
#Nos conectamos al sistema de archivos de S3
fs = s3fs.S3FileSystem()

In [9]:
#Listamos los archivos 
fs.glob(f"s3://{bucket}/data/insurance_dataset/*")

['datasetsbdajac/data/insurance_dataset/part-00000-9e9d29dc-a2ad-42df-bdfa-aa417ed32945-c000.csv',
 'datasetsbdajac/data/insurance_dataset/part-00001-9e9d29dc-a2ad-42df-bdfa-aa417ed32945-c000.csv',
 'datasetsbdajac/data/insurance_dataset/part-00002-9e9d29dc-a2ad-42df-bdfa-aa417ed32945-c000.csv',
 'datasetsbdajac/data/insurance_dataset/part-00003-9e9d29dc-a2ad-42df-bdfa-aa417ed32945-c000.csv']

In [10]:
#Colocamos los archivos dentro de una variable
archivos = fs.glob(f"s3://{bucket}/data/insurance_dataset/*")

In [11]:
#Variable en donde acumularemos los registros
#Creamos un dataframe vacío
dfpRegistros = pd.DataFrame()

In [12]:
#Iteramos cada archivo
for archivo in archivos:
    dfPRegistrosDeArchivo = pd.read_csv(f"s3://{archivo}", header = None)

    #Verificamos si estamos con el primer archivo
    if dfpRegistros.empty == True:
        #Inicializamos la variable que acumula los registros
        dfpRegistros = dfPRegistrosDeArchivo
    else:
        #Acumulamos los registros
        dfpRegistros = pd.concat(
            [dfpRegistros, dfPRegistrosDeArchivo],
            ignore_index = True #Ignoramos la columna índices que crea Pandas
        )

In [13]:
#Verificamos
dfpRegistros

,0,1,2,3,4,5,6,7,8,9,10,11
0,16884.92400,19.0,27.900,0.0,1,0,0,0,0,1,0,1
1,1725.55230,18.0,33.770,1.0,0,1,0,0,1,0,1,0
2,4449.46200,28.0,33.000,3.0,0,1,0,0,1,0,1,0
3,21984.47061,33.0,22.705,0.0,0,1,0,1,0,0,1,0
4,3866.85520,32.0,28.880,0.0,0,1,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1333,10600.54830,50.0,30.970,3.0,0,1,0,1,0,0,1,0
1334,2205.98080,18.0,31.920,0.0,1,0,1,0,0,0,1,0
1335,1629.83350,18.0,36.850,0.0,1,0,0,0,1,0,1,0
1336,2007.94500,21.0,25.800,0.0,1,0,0,0,0,1,1,0


In [14]:
#Como este archivo de casos tiene la columna que vamos a predecir, ignoramos el último campo
del dfpRegistros[0]

#Verificamos
dfpRegistros

,1,2,3,4,5,6,7,8,9,10,11
0,19.0,27.900,0.0,1,0,0,0,0,1,0,1
1,18.0,33.770,1.0,0,1,0,0,1,0,1,0
2,28.0,33.000,3.0,0,1,0,0,1,0,1,0
3,33.0,22.705,0.0,0,1,0,1,0,0,1,0
4,32.0,28.880,0.0,0,1,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
1333,50.0,30.970,3.0,0,1,0,1,0,0,1,0
1334,18.0,31.920,0.0,1,0,1,0,0,0,1,0
1335,18.0,36.850,0.0,1,0,0,0,1,0,1,0
1336,21.0,25.800,0.0,1,0,0,0,0,1,1,0


In [15]:
#Convertimos el dataframe a una matriz
#Utilitario para manipular arrays y matrices
import numpy as np

In [16]:
#Convertimos el dataframe a una matriz
matrizDeRegistros = dfpRegistros.values

#Verificamos
print(matrizDeRegistros)

[[19.   27.9   0.   ...  1.    0.    1.  ]
 [18.   33.77  1.   ...  0.    1.    0.  ]
 [28.   33.    3.   ...  0.    1.    0.  ]
 ...
 [18.   36.85  0.   ...  0.    1.    0.  ]
 [21.   25.8   0.   ...  1.    1.    0.  ]
 [61.   29.07  0.   ...  0.    0.    1.  ]]


# 5. Uso del Modelo

In [17]:
#Endpoint de acceso al modelo
nombreDelEndpoint = "endpoint-numerico-27-09-2024"

In [18]:
#Utilitario para usar el modelo
from sagemaker.predictor import Predictor

In [19]:
#Utilitario para serializar el INPUT del modelo (CSV)
from sagemaker.serializers import CSVSerializer

In [20]:
#Utilitario para serializar el OUTPUT del modelo (JSON)
from sagemaker.deserializers import JSONDeserializer

In [21]:
#Creamos un predictor para el modelo desplegado
predictor = Predictor(
    endpoint_name = nombreDelEndpoint, #Nombre del endpoint
    sagemaker_session = sesion, #Sesión de SageMaker
    serializer = CSVSerializer(), #Serializador que envía los datos al modelo
    deserializer = JSONDeserializer() #Des-serializador que extrae la respuesta del modelo
)

In [22]:
#Serializamos los registros
registrosSerializados = CSVSerializer().serialize(matrizDeRegistros)

In [23]:
#Usamos el modelo para hacer predicciones
resultados = predictor.predict(registrosSerializados)

In [24]:
#Verificamos los resultados
#Notemos que como no hemos calibrado el modelo, el modelo está entregando malos resultados
resultados

{'predictions': [{'score': 28349.5546875},
  {'score': 2490.05908203125},
  {'score': 5593.587890625},
  {'score': 3312.09912109375},
  {'score': 4345.48974609375},
  {'score': 2745.84326171875},
  {'score': 8287.255859375},
  {'score': 6009.54345703125},
  {'score': 5994.85693359375},
  {'score': 9586.849609375},
  {'score': 1526.6905517578125},
  {'score': 39029.78125},
  {'score': 2334.343017578125},
  {'score': 11462.15234375},
  {'score': 34781.8203125},
  {'score': -143.19384765625},
  {'score': 8558.763671875},
  {'score': 573.291259765625},
  {'score': 11237.9794921875},
  {'score': 33094.7890625},
  {'score': 11030.4912109375},
  {'score': 3358.187255859375},
  {'score': 2092.697265625},
  {'score': 33540.015625},
  {'score': 6265.9013671875},
  {'score': 11074.919921875},
  {'score': 9079.93359375},
  {'score': 10773.6669921875},
  {'score': 354.61474609375},
  {'score': 34463.125},
  {'score': 31291.22265625},
  {'score': -748.72900390625},
  {'score': 1874.465576171875},
  